In [ ]:
import cv2
import tensorflow.keras as keras
import tensorflow as tf
import math
import numpy as np
import os
import cv2
from google.colab.patches import cv2_imshow
import random
import copy
 
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd  
 
from numpy import unique
from numpy import where
from sklearn.datasets import make_classification
from sklearn.cluster import AffinityPropagation, DBSCAN, KMeans, AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
len_single = 5
len_pair = 1

In [ ]:
## belief vectors
BV12 = tf.Variable(np.random.randn(1,len_pair),trainable = True, dtype = tf.float64)
BV13 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)
BV14 = tf.Variable(np.random.randn(1,len_pair), trainable = True, dtype = tf.float64)
BV23 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)
BV24 = tf.Variable(np.random.randn(1,len_pair), trainable = True, dtype = tf.float64)
BV34 = tf.Variable(np.random.randn(1,len_pair ), trainable = True, dtype = tf.float64)

In [ ]:
 
#input
folder = '2-d_colliding'
#/content/drive/My Drive/VIPUL COLAB AND DATA/vipul_data
#path = '/content/drive/My Drive/VIPUL COLAB AND DATA/'
arr = np.load('/content/drive/My Drive/VIPUL COLAB AND DATA/collision_and_pass_data/' + folder +'/0.npy')
pos_arr = arr[3:, 0:8]
vel_arr = arr[3:, 8:16]
 
 
  
pos__1 = pos_arr[:,0:2]
pos__2 = pos_arr[:,2:4]
pos__3 = pos_arr[:,4:6]
pos__4 = pos_arr[:,6:8]
 
vel__1 = vel_arr[:,8:10]
vel__2 = vel_arr[:,10:12]
vel__3 = vel_arr[:,12:14]
vel__4 = vel_arr[:,14:16]
 
print( pos__1.shape,   pos_arr.shape,  vel__1.shape)
print(arr[3:])

In [ ]:
def distance(a, b):
  return np.sqrt(np.sum((a-b)**2))

In [ ]:
def rel_vel_mag(vel_a,vel_b):
  return np.sqrt(np.sum((vel_a-vel_b)**2))
  

In [ ]:
lst = [[], [], [], [], [], []]
pairs = [(1,2), (1,3), (1,4), (2,3), (2,4), (3,4)]
r = 1.5
dist_for_min = 2*r + 0.1
dist = 4.2
n = 6
for i in range(n):
  start = -1
  end = 1
  interval = [0,0]
  j = 0
  min_dist = 30
  while(j < pos__1.shape[0]):
    if(start == -1 and distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]) < dist):
      print(i,j,  "start started")
      interval[0] = j
      start = 1
      end = -1
      min_dist = 30
    elif (start == 1 and end == -1):
      if(distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]) > dist):
        if(min_dist < dist_for_min):
          print(i, j, "exited from start, and saved")
          #save and set start = -1, end = 1
          interval[1] = j;
          lst[i].append(interval)
          interval = [0,0]
          start = -1
          end = 1
        else:
          #dont save and set start to -1, end = 1
          print(i, j, "exited from start, not saved")
          start = -1
          end = 1
          interval = [0,0]
      else:
        min_dist = min(min_dist, distance(pos_arr[j,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[j, 2*pairs[i][1]-2 : 2*pairs[i][1]]))
    j += 1
 
 
print(lst)
 
popable = []
for i in range(len(lst)):
  for j in range(len(lst[i])):
    if(lst[i][j][0] == 0) :
      popable.append(i)
for i in popable:
  lst[i].pop(0)
print(lst)

In [ ]:
for i in range(6):
  for j in range(len(lst[i])):
    print("/////////////////////////////////////////////////",i , "///////////////////////", j )
    start = lst[i][j][0]
    end = lst[i][j][1]
    print([start, end])
    for k in range(start, end):
      print(distance(pos_arr[k,2*pairs[i][0]-2 : 2*pairs[i][0]],  pos_arr[k, 2*pairs[i][1]-2 : 2*pairs[i][1]]))

In [ ]:
X_indices = []

j_s = [0,1,3]
for j in j_s:
  ln = len(lst[j])
  for k1 in range(ln):
    for k2 in range(ln):
      print(k1, k2)
      temp = ((j,k1), (j,k2))
      X_indices.append(temp)
print(len(X_indices))

X_triplet = []
for i in j_s:
  for j in j_s:
    if(i == j):
      continue
    
    for ik1_1 in range(len(lst[i])):
      for ik1_2 in range(len(lst[i])):
        for ik2_1 in range(len(lst[i])):
          for jk2_1 in range(len(lst[j])):
            for jk3_1 in range(len(lst[j])):
              for jk3_2 in range(len(lst[j])):
                temp = (((i, ik1_1), (i, ik1_2)), ((i, ik2_1), (j, jk2_1)), ((j, jk3_1), (j, jk3_2)))
                print(temp)
                X_triplet.append(temp)
print(len(X_triplet))

In [ ]:
def similarity_model():
  inp1 = keras.Input(shape = (None, 2))
  inp2 = keras.Input(shape = (None, 2))

  encoder = keras.layers.LSTM(20)

  encoded1 = encoder(inp1)
  encoded2 = encoder(inp2)

  concat = keras.layers.Concatenate(axis = -1)([encoded1, encoded2])

  out = keras.layers.Dense(20, activation = 'sigmoid')(concat)
  out = keras.layers.Dense(15,  activation = 'sigmoid')(out)
  out = keras.layers.Dense(10,  activation = 'sigmoid')(out)
  out = keras.layers.Dense(5,  activation = 'sigmoid')(out)
  out = keras.layers.Dense(1,  activation = 'sigmoid')(out)
  model = keras.Model([inp1, inp2], out)
  return model

In [ ]:
s_m = similarity_model()
s_m.summary()

In [ ]:
def loss_fn(y_true, y_pred):
  loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
  #print("Magnitude loss :", loss)
  return loss

In [ ]:
def window_generator(start , finish, pair):
    #abs_pos
    inp_pos_first = pos_arr[start:finish, 2*pair[0]-2:2*pair[0]]
    inp_pos_second = pos_arr[start:finish, 2*pair[1]-2:2*pair[1]]
    rel_pos = np.subtract(inp_pos_first  ,inp_pos_second)
    abs_rel_dist = np.sqrt(np.sum((np.subtract(inp_pos_first  ,inp_pos_second))**2, axis = 1, keepdims = True))
    #abs vel
    inp_vel_first = vel_arr[start:finish, 2*pair[0]-2:2*pair[0]]
    inp_vel_second = vel_arr[start:finish, 2*pair[1]-2:2*pair[1]]
    rel_vel = np.subtract(inp_vel_first  ,inp_vel_second)
    abs_rel_vel = np.sqrt(np.sum((np.subtract(inp_vel_first  ,inp_vel_second))**2, axis = 1, keepdims = True))
    
    len = rel_vel.shape[0]
    #dv
    dv = np.zeros((rel_vel.shape))
    for i in range(len-1):
      dv[i] = rel_vel[i+1]-rel_vel[i]
    
    #abs_dv
    abs_dv = np.sqrt(np.sum((dv)**2, axis = 1, keepdims = True))

    #theta_v_x 
    theta_v_x = np.zeros((len, 1))
    for i in range(theta_v_x.shape[0]):  
      v0 = rel_pos[i]
      v1 = rel_vel[i]
      angle = np.arctan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
      theta_v_x[i,0] = np.degrees(angle)
    
    
    #theta_dv_x
    theta_dv_x = np.zeros((len, 1))
    for i in range(theta_dv_x.shape[0]):  
      v0 = rel_pos[i]
      v1 = dv[i]
      angle = np.arctan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
      theta_v_x[i,0] = np.degrees(angle)
    

    inp = np.concatenate((abs_rel_dist[:len-1,:], abs_rel_vel[:len-1,:], theta_v_x[:len-1,:], abs_dv[:len-1,:], theta_dv_x[:len-1,:]), axis = 1)
    
    return inp

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate = 0.001)
for i in range(500):
  print('iteration ;:::: ', i)
  loss1 = 0
  loss2 = 0
  with tf.GradientTape() as Tape:
    for pts in X_indices:
      j_k = pts[0]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp1 = data[:, 3:]
      inp1 = np.expand_dims(inp1, axis = 0)

      j_k = pts[1]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp2 = data[:, 3:]
      inp2 = np.expand_dims(inp2, axis = 0)

      exp_out = np.ones((1,1))
      
      out = s_m([inp1, inp2])

      #print(pts,  out)
      loss_temp = loss_fn(exp_out, out)
      loss1 += loss_temp
    #loss1 = loss1/len(X_indices)
    for pts in X_triplet:
      ##0
      j_k = pts[0][0]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp1 = data[:, 3:]
      inp1 = np.expand_dims(inp1, axis = 0)
      j_k = pts[0][1]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp2 = data[:, 3:]
      inp2 = np.expand_dims(inp2, axis = 0)
      exp_out = np.ones((1,1))
      out1 = s_m([inp1, inp2])
      ##1
      j_k = pts[1][0]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp1 = data[:, 3:]
      inp1 = np.expand_dims(inp1, axis = 0)
      j_k = pts[1][1]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp2 = data[:, 3:]
      inp2 = np.expand_dims(inp2, axis = 0)
      exp_out = np.ones((1,1))
      out2 = s_m([inp1, inp2])
      ##2
      j_k = pts[2][0]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp1 = data[:, 3:]
      inp1 = np.expand_dims(inp1, axis = 0)
      j_k = pts[2][1]
      j = j_k[0]
      k = j_k[1]
      start = lst[j][k][0]
      end = lst[j][k][1]
      data = window_generator(start, end+1, pairs[j])
      inp2 = data[:, 3:]
      inp2 = np.expand_dims(inp2, axis = 0)
      exp_out = np.ones((1,1))
      out3 = s_m([inp1, inp2])
      
      loss_temp = 2*out2[0,0] - out1[0,0] - out3[0,0]
      loss2 += loss_temp
    #loss2 = loss2/len(X_triplet)
    print(loss1, loss2)
    loss = loss1+loss2
  print("INFO :: " , loss)
  grad = Tape.gradient(loss, s_m.trainable_variables)
  optimizer.apply_gradients(zip(grad, s_m.trainable_variables))
  
  s_m.save('/content/drive/My Drive/VIPUL COLAB AND DATA/' + 's_m' + marker + '.hdf5')

In [ ]:
pts_lst = [((4,0), (0,0)), ((4,0), (1,0)) , ((4,0), (2,0)), ((4,0), (3,0)), ((4,0), (4,0)), ((4,0), (5,0))]
for pts in pts_lst:
  j_k = pts[0]
  j = j_k[0]
  k = j_k[1]
  start = lst[j][k][0]
  end = lst[j][k][1]
  data = window_generator(start, end+1, pairs[j])
  inp1 = data[:, 3:]
  inp1 = np.expand_dims(inp1, axis = 0)

  j_k   = pts[1]
  j     = j_k[0]
  k     = j_k[1]
  start = lst[j][k][0]
  end   = lst[j][k][1]
  data  = window_generator(start, end+1, pairs[j])
  inp2  = data[:, 3:]
  inp2  = np.expand_dims(inp2, axis = 0)

  exp_out = np.ones((1,1))

  out = s_m([inp1, inp2])

  print(pts,  out)